In [1]:
!pip install findspark

import findspark
findspark.init()
from pyspark.sql import SparkSession

from pyspark.conf import SparkConf
from pyspark import SparkContext


sc = SparkContext('local')

spark = SparkSession(sc)
sc

<SparkContext master=local appName=pyspark-shell>

In [2]:
train_df= spark.read.csv('adult.data',header=True , inferSchema = True, sep = ",")

In [3]:
test_df= spark.read.csv('adult.test',header=True , inferSchema = True, sep = ",")

In [4]:
train_df.show (5)

+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
|age|        workclass|  fnlwgt| education|education_num|     marital_status|        occupation|  relationship|  race|    sex|capital_gain|capital_loss|hours_per_week|native_country|output|
+---+-----------------+--------+----------+-------------+-------------------+------------------+--------------+------+-------+------------+------------+--------------+--------------+------+
| 39|        State-gov| 77516.0| Bachelors|         13.0|      Never-married|      Adm-clerical| Not-in-family| White|   Male|      2174.0|         0.0|          40.0| United-States| <=50K|
| 50| Self-emp-not-inc| 83311.0| Bachelors|         13.0| Married-civ-spouse|   Exec-managerial|       Husband| White|   Male|         0.0|         0.0|          13.0| United-States| <=50K|
| 38|          Private|215646.0|   HS-grad|       

DF çok geniş olduğu için düzgün bir görünüm alamadık. Driver'a aldığımız birkaç satırdan oluşan dataframe'i pandas dataframe'e çevirerek daha güzel bir görünüm sağlayabiliriz.

In [5]:
train_df.limit(5).toPandas().head(5)

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [6]:
test_df.limit(5).toPandas().head(5)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,output
0,25,Private,226802.0,11th,7.0,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K.
1,38,Private,89814.0,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K.
2,28,Local-gov,336951.0,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K.
3,44,Private,160323.0,Some-college,10.0,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K.
4,18,?,103497.0,Some-college,10.0,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K.


### Veri Birleştirme

In [7]:
# bu ikisinin şemaları aynı olduğu için birleştirebiliriz
adult_whole_df = train_df.union(test_df)
adult_whole_df.limit(5).toPandas().head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,output
0,39,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [8]:
print("train satır sayısı: ", train_df.count())
print("test satır sayısı:  ", test_df.count())
print("whole satır sayısı: ", adult_whole_df.count())

train satır sayısı:  32561
test satır sayısı:   16281
whole satır sayısı:  48842


Satır sayıları toplamda tutarlı olduğu için başlık yanlışlıkla whale_df içine girmemiş. Aferin union'a.

##VERİ SETİNİ İNCELEME ŞEMA İLE KARŞILAŞTIRMA

## VERİ TÜRLERİNİN INT VE STRING OLARAK DOĞRU BİR ŞEKİLDE GİRMİŞ Mİ KONTROL EDİYORUZ

In [9]:
adult_whole_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: double (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: double (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: double (nullable = true)
 |-- capital_loss: double (nullable = true)
 |-- hours_per_week: double (nullable = true)
 |-- native_country: string (nullable = true)
 |-- output: string (nullable = true)



ŞEMADA SIKINTI YOK

## NÜMERİK DEĞİŞKENLERİN İNCELENMESİ 

In [10]:
#adult_whole veri seti içinden tüm nümerik değişkenleri teker teker elle yazarak betimsel istatistiğine bakıyoruz.
adult_whole_df.describe(['age','fnlwgt','education_num','capital_gain','capital_loss','hours_per_week']) \
.toPandas().head()

,summary,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
0,count,48842,48842,48842,48842,48842,48842
1,mean,38.64358543876172,189664.13459727284,10.078088530363212,1079.0676262233324,87.50231358257237,40.422382375824085
2,stddev,13.710509934443563,105604.02542315733,2.570972755592263,7452.019057655401,403.0045521243599,12.3914440242523
3,min,17,12285.0,1.0,0.0,0.0,1.0
4,max,90,1490400.0,16.0,99999.0,4356.0,99.0


## KATEGORİK DEĞİŞKENLERİN İNCELENMESİ

In [11]:
#Kategorik değişkenlerin incelenmesinde groupBy() kullanmak daha çok bilgi verir. 
#Hedef değişken dahil toplam 9 kategorik değişken var.printSchema() sonuçlarına baktıktan sonra örneğin workclass kategorik
#değişkenin seçtik:


In [12]:
from pyspark.sql import functions as F

### Workclass'ı inceleyelim

In [13]:
#workclass'ı seçtik,
#agg fonrkisyonu olarak sql'dilindeiki yıldız(*)  ile işaretik böylece bütün sütunları count edeceğiz yani saydıracak
#sonra da pandas'a aktatıp ilk 10 satırı göstereceğiz:
# bu işlemleri teker teker tüm kategorik değişkenlere, onları incelemek üzere uygulamalıyız.

adult_whole_df.groupBy(F.col("workclass")) \
.agg({"*":"count"}) \
.toPandas().head(10)

,workclass,count(1)
0,State-gov,1981
1,Federal-gov,1432
2,Self-emp-not-inc,3862
3,Local-gov,3136
4,Private,33906
5,?,2799
6,Self-emp-inc,1695
7,Without-pay,21
8,Never-worked,10


## Education incleyelim

Yorum: 2.799 adet ? var. Bu nedir. Muhtelemen kayıp bilgi.
Daha sonra never-worked ve without-pay sınıfları çok az tekrarlanmış. Bunların da veri setinden çıkarılması düşünebilir.

In [14]:
adult_whole_df.groupBy(F.col("education")) \
.agg({"*":"count"}) \
.toPandas().head(20)

,education,count(1)
0,Prof-school,834
1,10th,1389
2,7th-8th,955
3,5th-6th,509
4,Assoc-acdm,1601
5,Assoc-voc,2061
6,Masters,2657
7,12th,657
8,Preschool,83
9,9th,756


Yorum: Genel bir sıkıntı görünmüyor ancak çok fazla kategori var belki bazıları birleştirilebilir.

      1st-4th, 5th-6th, 7th-8th: elementary-school

      9th, 10th, 11th, 12th: high-school

      Masters, Doctorate: high-education

      Bachelors, Some-college: undergraduate

/////////////////////// VERİ TEMİZLİĞİ İÇİN TAVSİYELER ////////////////////////////////////////////////

  1. Tüm sütunları boşluk kontrolü yap.
  2. ? içeren workclass, occupation  var bunların ? içerdiği satırlar tekrar incelenmeli.
      ? işaretleri sistematik bir şekilde mi oluşmuş yoksa bu oluşum tesadüfi mi?
      ? kayıtlarının oluşması altında yatan bir mekanizma var mı?
      Bu sistematik hata yakalanırsa veri doldurma (imputation) yoksa satır silme yapılsın.
  3. workclass niteliğinde never-worked ve without-pay sınıfları ve
      occupation niteliğinde  Armed-Forces  sınıfı
    çok az tekrarlanmış. Veri setinden çıkarılabilir.
  4. education niteliğindeki:
          1st-4th, 5th-6th, 7th-8th: elementary-school
          9th, 10th, 11th, 12th: high-school
          Masters, Doctorate: high-education
          Bachelors, Some-college: undergraduate
     sınıfları yukarıdaki gibi birleştirilebilir.
  5. output (hedef değişkendeki) >50K. ve <=50K. değerlerindeki "." kaldırılmalı

 # ==== VERİ TEMİZLİĞİ BAŞLIYOR ===========

## 1. Tüm sütunları boşluk kontrolü yap.

In [15]:


kategorik_nitelikler = ["workclass", "education", "marital_status", "occupation", "relationship", "race",
          "sex", "native_country", "output"]
numerik_nitelikler = ["age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week"]

In [16]:
#functions import edelim
from pyspark.sql.functions import *

In [17]:
#Boşluk temizleme (trim) sadece string değişkenlere uygulanabilir.
 
adult_whole_df1 = adult_whole_df \
.withColumn("workclass", F.trim(F.col("workclass"))) \
.withColumn("education", F.trim(F.col("education"))) \
.withColumn("marital_status", F.trim(F.col("marital_status"))) \
.withColumn("occupation", F.trim(F.col("occupation"))) \
.withColumn("relationship", F.trim(F.col("relationship"))) \
.withColumn("race", F.trim(F.col("race"))) \
.withColumn("sex", F.trim(F.col("sex"))) \
.withColumn("native_country", F.trim(F.col("native_country"))) \
.withColumn("output", F.trim(F.col("output")))

In [18]:
adult_whole_df.count()

48842

In [19]:
adult_whole_df1.count()

48842

# 2. OUTPUT ADINDAKİ HEDEF DEĞİŞKEN İÇİNDEKİ "." TEMİZLİĞİ
#### output isimli (hedef değişkendenin) >50K. ve <=50K. değerlerinin içindeki nokta "." kaldırılmalı


In [20]:
adult_whole_df2 = adult_whole_df1 \
.withColumn("output", regexp_replace(col("output"), "<=50K.","<=50K")) \
.withColumn("output", regexp_replace(col("output"), ">50K.",">50K"))

In [21]:
# Temizlik sonucunu görelim.
adult_whole_df2.groupBy(col("output")).agg({"*":"count"}) \
.toPandas().head()

,output,count(1)
0,<=50K,37155
1,>50K,11687


# 3. NULL KONTROLÜ

// Her bir sütun için for dongüsü ile o sütunda filter() metodu ile null kontrolü yapalım ve bunları sayalım. Eğer o sütunda en az bir adet null varsa yani >0 ise o sütunda null olduğunu ekrana yazdıralım.

In [22]:
adult_whole_df2.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education_num',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'output']

In [23]:
sayac_for_null = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).isNull()).count() > 0):
        print(sayac_for_null, ". ", sutun, " içinde null değer var.")
    else:
        print(sayac_for_null, ". ", sutun)
    sayac_for_null+=1

1 .  age
2 .  workclass
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country
15 .  output


// Herhangi bir null yok.

# 4. "?" KONTROLLERİ

In [24]:
sayac_for_question = 1
for sutun in adult_whole_df2.columns:
    if(adult_whole_df2.filter(col(sutun).contains("?")).count() > 0):
        print(sayac_for_question, ". ", sutun, " içinde ? var.")
    else:
        print(sayac_for_question, ". ", sutun)
    sayac_for_question+=1

1 .  age
2 .  workclass  içinde ? var.
3 .  fnlwgt
4 .  education
5 .  education_num
6 .  marital_status
7 .  occupation  içinde ? var.
8 .  relationship
9 .  race
10 .  sex
11 .  capital_gain
12 .  capital_loss
13 .  hours_per_week
14 .  native_country  içinde ? var.
15 .  output


// workclass, occupation, native_country içinde ? var.

// Bunların hedef değişken output ile ilişkisini inceleyelim.

In [25]:
adult_whole_df2.select("workclass", "occupation", "native_country", "output") \
.filter(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")) \
.groupBy("workclass", "occupation", "native_country", "output").count() \
.orderBy(col("count").desc()) \
.toPandas().head(50)

,workclass,occupation,native_country,output,count
0,?,?,United-States,<=50K,2284
1,?,?,United-States,>50K,246
2,Private,Other-service,?,<=50K,100
3,Private,Sales,?,<=50K,55
4,Private,Prof-specialty,?,<=50K,51
5,Private,Craft-repair,?,<=50K,48
6,Private,Prof-specialty,?,>50K,48
7,?,?,Mexico,<=50K,48
8,Private,Adm-clerical,?,<=50K,47
9,Private,Machine-op-inspct,?,<=50K,42


// Soru işaretlerinin dağılımı ve hedef değişken ile ilgisi tesadüfi görünüyor. Bu durumda ? işareti içeren satırları veri setinden çıkaralım.

In [26]:
adult_whole_df3 = adult_whole_df2 \
        .filter(~(col("workclass").contains("?") | col("occupation").contains("?") | col("native_country").contains("?")))

In [27]:
print(adult_whole_df2.count())
print(adult_whole_df3.count())

48842
45222


// adult_whole_df2'den adult_whole_df3'e geçerken ? işareti olan satırlar silindi.

# 5. ZAYIF SINIFLARIN KALDIRILMASI
 // workclass niteliğinde never-worked ve without-pay sınıfları ve
occupation niteliğinde  Armed-Forces  sınıfı
  çok az tekrarlanmış. Veri setinden çıkarılabilir.

In [28]:
adult_whole_df4 = adult_whole_df3 \
.filter(~(col("workclass").contains("never-worked") | col("workclass").contains("without-pay") |
          col("occupation").contains("Armed-Forces")))

In [29]:
print(adult_whole_df3.count())
print(adult_whole_df4.count())

45222
45208


/ adult_whole_df3'ten adult_whole_df4'e geçerken zayıf sınıfların bulunduğu satırlar silindi.

# 6. EĞİTİM DURUMUYLA İLGİLİ KATEGORİLERİ BİRLEŞTİRME
// education niteliğindeki:
  1st-4th, 5th-6th, 7th-8th: elementary-school
9th, 10th, 11th, 12th: high-school
Masters, Doctorate: high-education
Bachelors, Some-college: undergraduate
sınıfları yukarıdaki gibi birleştirilebilir.

 ## Uyarı: Aşağıdaki yöntem trim edilmemiş niteliklerde kullanılmamalıdır.

In [30]:
adult_whole_df5 = adult_whole_df4.withColumn("education_merged", \
      when(col("education").isin("1st-4th","education","5th-6th","education","7th-8th"), "Elementary-School")
        .when(col("education").isin("9th","10th","11th","12th"), "High-School")
        .when(col("education").isin("Masters","Doctorate"), "Post-Graduate")
        .when(col("education").isin("Bachelors","Some-college"), "Under-Graduate")
        .otherwise(col("education")))
adult_whole_df5.select("education","education_merged").toPandas().head(20)

,education,education_merged
0,Bachelors,Under-Graduate
1,Bachelors,Under-Graduate
2,HS-grad,HS-grad
3,11th,High-School
4,Bachelors,Under-Graduate
5,Masters,Post-Graduate
6,9th,High-School
7,HS-grad,HS-grad
8,Masters,Post-Graduate
9,Bachelors,Under-Graduate


# 7. native_country niteliğinde temizlik
// native_country'de ? var ve Hollanda 1 kez tekrarlanmış.
// ? temizleme işini 4. aşamada hallettik.
// Burada sadece Hollandayı (Holand-Netherlands) çıkaralım.


In [31]:
adult_whole_df6 = adult_whole_df5.filter(~(col("native_country") == "Holand-Netherlands"))
print(adult_whole_df5.count())
print(adult_whole_df6.count())

45208
45207


Satır bir azaldığına göre çıkarma işlemi başarılı.

## 8. OUTPUT . KALDIRMA
 // Bu temizliği 2. aşamada yapmıştık.

# 9. TEMİZ VERİYİ DİSKE YAZALIM
// Sütun sırasını değiştirelim. education_merge'i education yanına alalım

öncelikle nitelik sıralama adında yeni bir liste yazarak seçmek istediğimiz sütünları sırasıyla yazıyoruz. kategorik değişkenleri en başta sol tarafa, numerikleri sağ tarafa, output değişkenin ise en sağa attık. daha sonra select ile bu listeyi seçerek yeni bir nesneye atama yapıyoruz.

In [32]:
nitelik_siralama = ["workclass", "education", "education_merged", "marital_status", "occupation", "relationship", "race",
     "sex", "native_country", "age", "fnlwgt", "education_num", "capital_gain", "capital_loss", "hours_per_week","output"]
adult_whole_df7 = adult_whole_df6.select(nitelik_siralama)

In [33]:
adult_whole_df7.toPandas().head()

,workclass,education,education_merged,marital_status,occupation,relationship,race,sex,native_country,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week,output
0,State-gov,Bachelors,Under-Graduate,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,39,77516.0,13.0,2174.0,0.0,40.0,<=50K
1,Self-emp-not-inc,Bachelors,Under-Graduate,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,50,83311.0,13.0,0.0,0.0,13.0,<=50K
2,Private,HS-grad,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,38,215646.0,9.0,0.0,0.0,40.0,<=50K
3,Private,11th,High-School,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,53,234721.0,7.0,0.0,0.0,40.0,<=50K
4,Private,Bachelors,Under-Graduate,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,28,338409.0,13.0,0.0,0.0,40.0,<=50K


yukırdaki gibi diske kaydedildi.


https://github.com/veribilimiokulu/udemy-apache-spark/blob/master/kurs_hazirlik/pyspark/MachineLearning/Preprocessing/DataCleaning.ipynb

In [34]:
sc.stop()